In [1]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
from shapely.geometry import MultiPoint
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt

/home/jerryluo8/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [ ]:
%%time
place = 'Chicago'
G_chc = ox.graph_from_place(place,network_type='walk', buffer_dist = 5000, simplify=False)

In [9]:
chc_polls = gpd.read_file('chc_polls.geojson')
chc_polls['longitude']

0     -87.989056
1     -88.002518
2     -88.039747
3     -88.044982
4     -87.985154
         ...    
451   -87.656478
452   -87.653913
453   -87.607469
454   -87.627398
455   -87.626297
Name: longitude, Length: 456, dtype: float64

In [5]:
chc_polls = gpd.read_file('chc_polls.geojson')
chc_nodes=ox.distance.nearest_nodes(G_chc,chc_polls.longitude,chc_polls.latitude)
chc_walk=np.full((len(chc_nodes),len(chc_nodes)),0)

NameError: name 'G_chc' is not defined

In [ ]:
%%time
pairs_list = []

for i in range(len(chc_nodes)):
    for j in range(i+1,len(chc_nodes)):
        pairs_list.append((i,j))
        
#         dist = nx.shortest_path_length(G_chc,chc_nodes[i],chc_nodes[j],weight='length')/1.42
#         chc_walk[i,j]=dist
#         chc_walk[j,i]=dist

In [ ]:
def walk_dist_compute(pair):
    return nx.shortest_path_length(G_chc,chc_nodes[pair[0]],chc_nodes[pair[1]],weight='length')/1.42

In [ ]:
import multiprocess as mp

pool = mp.Pool(60)
distances_list = pool.map(walk_dist_compute, pairs_list)
pool.close()

In [ ]:
for i in range(len(pairs_list)):
    chc_walk[pairs_list[i][0],pairs_list[i][1]]=distances_list[i]
    chc_walk[pairs_list[i][1],pairs_list[i][0]]=distances_list[i]

In [ ]:
np.save('chc_walk.npy',chc_walk)